In [35]:
import numpy as np
import pandas as pd
import matrix_factorization_utilities

In [36]:
rating1_df = pd.read_csv('movie_ratings_data_set.csv')

In [37]:
rating1_df.head()

,user_id,movie_id,value
0,1,28,4
1,1,26,4
2,1,9,4
3,1,1,4
4,1,14,4


In [38]:
movies_df=pd.read_csv('movies.csv', index_col='movie_id')

In [39]:
movies_df.head()

,title,genre
movie_id,,
1,The Sheriff 1,"crime drama, western"
2,The Big City Judge 1,legal drama
3,The Sheriff 2,"crime drama, western"
4,Just a Regular Family,reality
5,The Big City Judge 2,legal drama


In [40]:
rating_df= pd.pivot_table(rating1_df, index='user_id', columns='movie_id', aggfunc=np.max)

In [41]:
rating_df.head()

value                                          ...                \
movie_id    1    2    3    4    5   6   7   8    9   10 ...   25   26  27   
user_id                                                 ...                 
1          4.0  NaN  NaN  NaN  NaN NaN NaN NaN  4.0 NaN ...  NaN  4.0 NaN   
2          5.0  5.0  NaN  NaN  NaN NaN NaN NaN  NaN NaN ...  NaN  NaN NaN   
3          4.0  4.0  5.0  NaN  NaN NaN NaN NaN  NaN NaN ...  NaN  NaN NaN   
4          5.0  5.0  NaN  5.0  5.0 NaN NaN NaN  NaN NaN ...  NaN  NaN NaN   
5          5.0  NaN  NaN  NaN  NaN NaN NaN NaN  5.0 NaN ...  NaN  NaN NaN   

                                            
movie_id   28   29  30   31   32   33   34  
user_id                                     
1         4.0  NaN NaN  NaN  NaN  NaN  NaN  
2         NaN  NaN NaN  3.0  NaN  NaN  4.0  
3         NaN  NaN NaN  NaN  NaN  NaN  NaN  
4         NaN  NaN NaN  NaN  NaN  NaN  NaN  
5         NaN  3.0 NaN  3.0  2.0  5.0  5.0  

[5 rows x 34 columns]

In [42]:
U,M=matrix_factorization_utilities.low_rank_matrix_factorization(rating_df.as_matrix(),num_features=15,regularization_amount=0.1)

         Current function value: 32.504363
         Iterations: 3000
         Function evaluations: 4505
         Gradient evaluations: 4505


In [43]:
predicted_ratings = np.matmul(U, M)
M = np.transpose(M)

In [59]:
print("Enter user_id to get recomemndations (Between 1 and 100):")
user_id_to_search = int(input())

movie_id =5

Enter user_id to get recomemndations (Between 1 and 100):
2


In [60]:
print("Movies previously reviewed by user_id {}:".format(user_id_to_search))
reviewed_movies_df = rating1_df[rating1_df['user_id']== user_id_to_search]
reviewed_movies_df = reviewed_movies_df.join(movies_df, on='movie_id')

movie_information = movies_df.loc[5]

Movies previously reviewed by user_id 2:


In [61]:
print(reviewed_movies_df[['title', 'genre', 'value']])
input("Press Enter to continue.")
print("Moview we will recomend:")

print("We are finding movie simillar to this movie:")
print("Movie title: {}".format(movie_information.title))
print("Genre: {}".format(movie_information.genre))

                         title                     genre  value
6         The Big City Judge 1               legal drama      5
7   We Will Fight Those Aliens            sci-fi, action      4
8                The Sheriff 1      crime drama, western      5
9              Political Gaffs  comedy, political satire      5
10       The Serious Detective           detective drama      4
11              The Spy Family                 spy drama      4
12       My Complicated Family              comedy-drama      3
Press Enter to continue.
Moview we will recomend:
We are finding movie simillar to this movie:
Movie title: The Big City Judge 2
Genre: legal drama


In [62]:
user_ratings = predicted_ratings[user_id_to_search - 1]
movies_df['rating'] = user_ratings

current_movie_features = M[movie_id - 1]

In [63]:
print("The attributes for this movie are:")
print(current_movie_features)

The attributes for this movie are:
[ 1.01290407 -0.84690299 -0.75039295  0.63562955 -0.97885446 -1.6712716
 -0.95825561  0.28137912 -0.30292413 -0.05226884 -0.10080464 -0.24149324
 -0.03170635 -0.83671211  1.0796329 ]


In [64]:
already_reviewed = reviewed_movies_df['movie_id']
recommended_df = movies_df[movies_df.index.isin(already_reviewed)==False]
recommended_df = recommended_df.sort_values(by=['rating'], ascending = False)

difference = M- current_movie_features

In [65]:
absolute_dufference =  np.abs(difference)

In [66]:
total_difference = np.sum(absolute_dufference, axis=1)

In [67]:
movies_df['difference_score']= total_difference

In [68]:
sort_movies_list= movies_df.sort_values('difference_score')

In [69]:
print(recommended_df[['title', 'genre', 'rating']].head(5),"\n","\n","\n")

print("The five most similar movies are:")
print(sort_movies_list[['title','difference_score']][1:6])

                            title                   genre    rating
movie_id                                                           
10        Surrounded by Zombies 1  horror, zombie fiction  4.837406
5            The Big City Judge 2             legal drama  4.831207
13                  The Sheriff 3    crime drama, western  4.776908
24           The Big City Judge 3             legal drama  4.732256
6               Attack on Earth 1          sci-fi, action  4.706611 
 
 

The five most similar movies are:
                             title  difference_score
movie_id                                            
10         Surrounded by Zombies 1          2.775594
8         Sci-Fi Murder Detectives          3.495761
9                      Biker Gangs          3.515093
3                    The Sheriff 2          4.038468
26               Mafia Underground          4.079803
